# 教程 2：任务生成和 CANN 仿真

> **阅读时间**：约 20-25 分钟
> **难度**：初级
> **先决条件**：[教程 1](./01_build_cann_model.ipynb)

本教程教您如何使用 Task 模块生成任务数据并使用 CANN 模型运行仿真。

---

## 目录

1. [任务模块概述](#1-task-module-overview)
2. [PopulationCoding1D 详解](#2-populationcoding1d-in-detail)
3. [使用 brainstate.for_loop 运行仿真](#3-running-simulations-with-brainstatefor_loop)
4. [完整示例](#4-complete-example)
5. [下一步](#5-next-steps)

---

## 1. 任务模块概述

CANNs Task模块生成实验范式和输入数据。Task与Model之间的关系：

- **Task**: 生成外部刺激序列（输入数据）
- **Model**: 消费输入数据，在仿真循环中运行

### 任务类别

CANNs提供两种主要任务类型：

**追踪任务**：
- `PopulationCoding1D/2D` - 群体编码
- `TemplateMatching1D/2D` - 模板匹配
- `SmoothTracking1D/2D` - 平滑追踪

**导航任务**：
- `ClosedLoopNavigation` - 闭环导航
- `OpenLoopNavigation` - 开环导航

> 本教程使用最简单的 `PopulationCoding1D` 作为示例。其他任务遵循类似的使用模式，但初始化参数不同。我们将在后续教程中演示不同的任务。

---

## 2. 详细的PopulationCoding1D

`PopulationCoding1D` 是一个简单的群体编码任务：无刺激 → 刺激 → 无刺激。这测试网络形成和维持记忆凸起的能力。

### 2.1 导入和创建任务

In [7]:
from canns.task.tracking import PopulationCoding1D
from canns.models.basic import CANN1D
import brainstate

# 首先创建模型实例
brainstate.environ.set(dt=0.1)
model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)
model.init_state()

# 创建任务
task = PopulationCoding1D(
    cann_instance=model,      # CANN模型实例
    before_duration=10.0,     # 刺激前的持续时间
    after_duration=50.0,      # 刺激后的持续时间
    Iext=0.0,                 # 特征空间中的刺激位置
    duration=10.0,            # 刺激持续时间
    time_step=0.1,            # 时间步长
)

### 2.2 参数说明

| 参数 | 类型 | 说明 |
|-----------|------|-------------|
| `cann_instance` | BaseCANN1D | CANN模型实例，任务通过调用其`get_stimulus_by_pos()`方法 |
| `before_duration` | float | 刺激呈现前的持续时间（无输入期间） |
| `after_duration` | float | 刺激结束后的持续时间（观察bump维持） |
| `Iext` | float | 特征空间中的刺激位置，通常在`[z_min, z_max]`范围内 |
| `duration` | float | 刺激呈现的持续时间 |
| `time_step` | float | 仿真时间步长，应与`brainstate.environ.set(dt=...)`匹配 |

**这些参数为什么重要**:
- `cann_instance`是必需的，因为任务需要调用模型的`get_stimulus_by_pos()`方法来生成合适的刺激
- `before_duration`和`after_duration`允许观察bump形成和维持
- `Iext`决定了bump形成的位置
- 所有持续时间使用与`time_step`相同的单位

### 2.3 获取任务数据

创建任务后，调用`get_data()`来生成并将输入数据存储在`task.data`中：

In [8]:
# 生成任务数据
task.get_data()

# 访问任务属性
print(f"总时间步数: {task.total_steps}")
print(f"总持续时间: {task.total_duration}")
print(f"数据形状: {task.data.shape}")

<PopulationCoding1D>Generating Task data(No For Loop)
Total time steps: 700
Total duration: 70.0
Data shape: (700, 256)


> **重要提示**：`get_data()` 不返回值。它就地修改 `task.data`。通过 `task.data` 访问数据。

---

## 3. 使用 brainstate.for_loop 运行模拟

### 3.1 为什么使用 for_loop？

BrainState 提供 `brainstate.transform.for_loop` 用于高效的模拟循环。与 Python 的 `for` 循环相比，它提供：

- **JIT 编译**：整个循环编译为高效的机器代码
- **GPU 加速**：自动 GPU 利用
- **自动向量化**：更好的内存访问模式

> **了解更多**：查看 [BrainState 循环教程](https://brainstate.readthedocs.io/tutorials/transforms/05_loops_conditions.html) 了解详细的 `for_loop` 使用方法。

### 3.2 基本用法

In [9]:
import brainstate
import brainunit as u

# 定义步骤函数
def run_step(t, inp):
    """
    单个仿真步骤。

    Args:
        t: 当前时间步索引
        inp: 当前时间步的输入数据

    Returns:
        要记录的状态变量
    """
    model(inp)  # 或 model.update(inp)
    return model.u.value, model.r.value

# 使用 task.data 运行仿真
results = brainstate.transform.for_loop(
    run_step,           # 步骤函数
    task.run_steps,     # 时间步数
    task.data,          # 输入数据（来自任务）
    pbar=brainstate.transform.ProgressBar(10)  # 可选的进度条
)

  0%|          | 0/700 [00:00<?, ?it/s]

### 3.3 处理返回值

`for_loop` 返回与步长函数返回对应的值：

In [10]:
# results是所有时间步长的返回值元组
u_history, r_history = results

print(f"膜电位历史形状: {u_history.shape}")  # (run_steps, num)
print(f"放电率历史形状: {r_history.shape}")  # (run_steps, num)

Membrane potential history shape: (700, 256)
Firing rate history shape: (700, 256)


### 3.4 JIT 编译的优势

首次运行包含编译时间（几秒钟），但后续运行快得多：

In [12]:
import time

# 第一次运行（包括编译）
start = time.time()
results = brainstate.transform.for_loop(run_step, task.run_steps, task.data)
print(f"第一次运行: {time.time() - start:.2f}s")

# 重新初始化状态
model.init_state()

# 第二次运行（已编译）
start = time.time()
results = brainstate.transform.for_loop(run_step, task.run_steps, task.data)
print(f"第二次运行: {time.time() - start:.2f}s")

First run: 0.63s
Second run: 0.11s


---

## 4. 完整示例

这是从模型创建到仿真的完整示例：

In [16]:
import brainstate
import brainunit as u
from canns.models.basic import CANN1D
from canns.task.tracking import PopulationCoding1D

# ============================================================
# 步骤 1: 设置环境并创建模型
# ============================================================
brainstate.environ.set(dt=0.1)

model = CANN1D(num=256, tau=1.0, k=8.1, a=0.5, A=10, J0=4.0)
model.init_state()

# ============================================================
# 步骤 2: 创建任务
# ============================================================
task = PopulationCoding1D(
    cann_instance=model,
    before_duration=10.0,
    after_duration=50.0,
    Iext=0.0,
    duration=10.0,
    time_step=0.1,
)

# 获取任务数据
task.get_data()

print("任务信息:")
print(f"  总时间步数: {task.total_steps}")
print(f"  总持续时间: {task.total_duration}")
print(f"  数据形状: {task.data.shape}")

# ============================================================
# 步骤 3: 定义模拟步骤函数
# ============================================================
def run_step(t, inp):
    model.update(inp)
    return model.u.value, model.r.value

# ============================================================
# 步骤 4: 运行模拟
# ============================================================
u_history, r_history = brainstate.transform.for_loop(
    run_step,
    task.run_steps,
    task.data,
)

# ============================================================
# 步骤 5: 检查结果
# ============================================================
print("\n模拟结果:")
print(f"  膜电位历史形状: {u_history.shape}")
print(f"  放电率历史形状: {r_history.shape}")

# 检查不同阶段的状态
before_steps = int(10.0 / 0.1)  # 刺激前
stim_end = int(20.0 / 0.1)      # 刺激结束
after_steps = int(70.0 / 0.1)   # 模拟结束

print(f"\n刺激前 (t={before_steps-1}) 最大放电率: {u.math.max(r_history[before_steps-1]):.6f}")
print(f"刺激期间 (t={stim_end-1}) 最大放电率: {u.math.max(r_history[stim_end-1]):.6f}")
print(f"刺激后 (t={after_steps-1}) 最大放电率: {u.math.max(r_history[after_steps-1]):.6f}")

<PopulationCoding1D>Generating Task data(No For Loop)
Task Information:
  Total time steps: 700
  Total duration: 70.0
  Data shape: (700, 256)

Simulation Results:
  Membrane potential history shape: (700, 256)
  Firing rate history shape: (700, 256)

Before stimulus (t=99) max firing rate: 0.000000
During stimulus (t=199) max firing rate: 0.002426
After stimulus (t=699) max firing rate: 0.002348


**预期输出**：
- 刺激前：放电速率~0
- 刺激期间：放电速率增加（凸起形成）
- 刺激后：放电速率保持（记忆持续）

---

## 5. 后续步骤

恭喜你完成了教程 2！你现在了解了：
- 如何使用 Task 模块生成数据
- PopulationCoding1D 的所有参数
- 如何使用 `brainstate.for_loop` 运行高效的模拟

### 继续学习

- **下一步**：[教程 3：分析和可视化](./03_analysis_visualization.ipynb) - 学习如何可视化模拟结果
- **其他任务类型**：我们将在教程 3 中演示不同任务的效果
- **更多关于 for_loop 的内容**：[BrainState 循环教程](https://brainstate.readthedocs.io/tutorials/transforms/05_loops_conditions.html)